In [398]:
%load_ext autoreload
%autoreload 2

import logging
from urllib.parse import urljoin
from io import StringIO
from typing import Sequence
from datetime import date, datetime

import requests
import pandas as pd
import numpy as np

from etf_scraper.base import SecurityListing
from etf_scraper.scrapers import ISharesListings, SSGAListings, InvescoListings, VanguardListings

logger = logging.getLogger(__name__)

il = InvescoListings.retrieve_listings()
van = VanguardListings.retrieve_listings()
ssga_listings = SSGAListings.retrieve_listings()        
ishares_listings = ISharesListings.retrieve_listings()

#TODO: Vanguard, Invesco - get proof of concept for one of ishares, ssga holdings first

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ].apply(parse_date)
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:318: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  )
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:113: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newva

In [410]:
all_listings = pd.concat([il, van, ssga_listings, ishares_listings])
all_listings.to_csv('./listings_2023_01_15.csv', index=False)

In [401]:
all_listings.isna().sum()

fund_name           0
ticker              0
inception_date      0
benchmark         897
cusip              69
isin              422
exchange          853
item_url          994
fund_type           0
provider            0
product_id        456
asset_class       315
subasset_class    456
product_url       246
region            662
net_assets        662
country           803
dtype: int64

In [393]:
il.tail(3)

,fund_name,ticker,inception_date,benchmark,cusip,isin,exchange,item_url,fund_type,provider
243,Invesco Ultra Short Duration ETF,GSY,2008-02-12,NaN,46090A887,US46090A8870,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco
244,Invesco VRDO Tax-Free ETF,PVI,2007-11-15,PVIX,46138G862,US46138G8621,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco
245,Invesco Variable Rate Investment Grade ETF,VRIG,2016-09-22,NaN,46090A879,US46090A8797,NASDAQ,https://www.invesco.com/us/financial-products/...,ETF,Invesco


In [394]:
ssga_listings.tail(3)

,fund_name,ticker,product_url,inception_date,region,net_assets,exchange,fund_type,asset_class,cusip,isin,benchmark,provider
207,The Real Estate Select Sector SPDR® Fund,XLRE,https://www.ssga.com/us/en/intermediary/etfs/f...,2015-10-07,US,5.356750e+09,NYSE ARCA EXCHANGE,ETF,Equity,81369Y860,US81369Y8600,Real Estate Select Sector Index,SSGA
208,The Technology Select Sector SPDR® Fund,XLK,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,3.930098e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y803,US81369Y8030,Technology Select Sector Index,SSGA
209,The Utilities Select Sector SPDR® Fund,XLU,https://www.ssga.com/us/en/intermediary/etfs/f...,1998-12-16,US,1.658127e+10,NYSE ARCA EXCHANGE,ETF,Equity,81369Y886,US81369Y8865,Utilities Select Sector Index,SSGA


In [395]:
van.tail(3)

,ticker,cusip,fund_name,inception_date,product_id,asset_class,subasset_class,fund_type,product_url,provider
344,VTWAX,922042619,Vanguard Total World Stock Index Fund Admiral ...,2019-02-07,5028,Stock Funds,International/Global Stock,MF,https://advisors.vanguard.com/investments/prod...,Vanguard
345,VTWIX,922042759,Vanguard Total World Stock Index Fund Institut...,2008-10-09,0826,Stock Funds,International/Global Stock,MF,https://advisors.vanguard.com/investments/prod...,Vanguard
346,VCMDX,921939708,Vanguard Commodity Strategy Fund Admiral Shares,2019-06-25,0517,Stock Funds,Specialty,MF,https://advisors.vanguard.com/investments/prod...,Vanguard


In [396]:
ishares_listings.tail(3)

,fund_name,inception_date,ticker,cusip,isin,asset_class,subasset_class,country,region,product_url,product_id,net_assets,fund_type,provider
434,iShares ESG Advanced MSCI EAFE ETF,2020-06-16,DMXF,46436E759,US46436E7590,Equity,Large Cap,Broad,Global,https://www.ishares.com/us/products/314362/ish...,314362,4.246446e+08,ETF,IShares
435,iShares MSCI Brazil ETF,2000-07-10,EWZ,464286400,US4642864007,Equity,Large/Mid Cap,Brazil,Latin America,https://www.ishares.com/us/products/239612/ish...,239612,4.972791e+09,ETF,IShares
436,iShares Municipal Bond Index Fund,2018-11-16,BIDPX,066923210,US0669232102,Fixed Income,Municipals,United States,North America,https://www.ishares.com/us/products/306171/ish...,306171,2.273509e+08,MF,IShares


In [378]:
# with open("test_SSGA_SPY_resp.xlsx", 'wb') as f:
#     f.write(spy.content)

spy = SSGAListings.retrieve_holdings_("SPY")
spy

/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/utils.py:51: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, col] = pd.to_numeric(df[col])


,name,ticker,cusip,sedol,weight,Sector,amount,local_currency,fund_ticker,as_of_date
4,Apple Inc.,AAPL,03783310,2046251,5.981867,Information Technology,166167790.0,USD,SPY,2023-01-12
5,Microsoft Corporation,MSFT,59491810,2588173,5.331199,Information Technology,82835550.0,USD,SPY,2023-01-12
6,Amazon.com Inc.,AMZN,02313510,2000019,2.535409,Consumer Discretionary,98625770.0,USD,SPY,2023-01-12
7,Berkshire Hathaway Inc. Class B,BRK.B,08467070,2073390,1.722998,Financials,20021100.0,USD,SPY,2023-01-12
8,Alphabet Inc. Class A,GOOGL,02079K30,BYVY8G0,1.632137,Communication Services,66373336.0,USD,SPY,2023-01-12
...,...,...,...,...,...,...,...,...,...,...
503,Lincoln National Corp,LNC,53418710,2516378,0.014596,Financials,1715566.0,USD,SPY,2023-01-12
504,Fox Corporation Class B,FOX,35137L20,BJJMGY5,0.013262,Communication Services,1624680.0,USD,SPY,2023-01-12
505,DaVita Inc.,DVA,23918K10,2898087,0.013018,Health Care,616770.0,USD,SPY,2023-01-12
506,DISH Network Corporation Class A,DISH,25470M10,2303581,0.011113,Communication Services,2734767.0,USD,SPY,2023-01-12


In [384]:
# with open("./test_IShares_IVV_holdings_current_resp.csv", 'wb') as f:
#     f.write(fhun_.content)

ivv = ishares_listings[ishares_listings['ticker']=="IVV"].iloc[0]
ivv_hist = ISharesListings.retrieve_holdings_(ivv['ticker'], ivv['product_url'], date(year=2022, month=10, day=31))
ivv_now = ISharesListings.retrieve_holdings_(ivv['ticker'], ivv['product_url'], None)

In [397]:
ivv_now

,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,AAPL,APPLE INC,Information Technology,Equity,1.787452e+10,5.97,1.787452e+10,133981885.0,133.41,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
1,MSFT,MICROSOFT CORP,Information Technology,Equity,1.593026e+10,5.32,1.593026e+10,66790747.0,238.51,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,7.576094e+09,2.53,7.576094e+09,79522351.0,95.27,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
3,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Financials,Equity,5.148520e+09,1.72,5.148520e+09,16143103.0,318.93,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2023-01-12
4,GOOGL,ALPHABET INC CLASS A,Communication,Equity,4.877005e+09,1.63,4.877005e+09,53517008.0,91.13,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,DISH,DISH NETWORK CORP CLASS A,Communication,Equity,3.387050e+07,0.01,3.387050e+07,2249037.0,15.06,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
505,NWS,NEWS CORP CLASS B,Communication,Equity,2.103345e+07,0.01,2.103345e+07,1066605.0,19.72,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2023-01-12
506,ETD_USD,ETD USD BALANCE WITH R93531,Cash and/or Derivatives,Cash,7.500000e+01,0.00,7.500000e+01,75.0,100.00,United States,NaN,USD,1.0,USD,"Jan 01, 2000",IVV,2023-01-12
507,MARGIN_USD,FUTURES USD MARGIN BALANCE,Cash and/or Derivatives,Cash Collateral and Margins,-7.500000e+01,0.00,-7.500000e+01,-75.0,100.00,United States,NaN,USD,1.0,USD,"Jan 01, 2000",IVV,2023-01-12


In [211]:
item_id = "0968"
voo = VanguardListings.retrieve_holdings_("VOO", item_id, date(year=2022, month=11, day=30))
voo

NameError: name 'VanguardListings' is not defined

In [391]:
# with open('./Invesco_QQQ_holdings_20230113.csv', 'wb') as f:
#     f.write(df)

qqq = InvescoListings.retrieve_holdings_("QQQ")
qqq

,fund_ticker,cusip,ticker,amount,market_value,weight,name,security_type,sector,as_of_date
0,QQQ,594918104,MSFT,74336015,1.778340e+10,11.905,Microsoft Corp,Common Stock,Information Technology,2023-01-13
1,QQQ,037833100,AAPL,128123489,1.726592e+10,11.559,Apple Inc,Common Stock,Information Technology,2023-01-13
2,QQQ,023135106,AMZN,102155587,1.002351e+10,6.710,Amazon.com Inc,Common Stock,Consumer Discretionary,2023-01-13
3,QQQ,02079K107,GOOG,60942986,5.655509e+09,3.786,Alphabet Inc,Common Stock,Communication Services,2023-01-13
4,QQQ,02079K305,GOOGL,60961599,5.615782e+09,3.759,Alphabet Inc,Common Stock,Communication Services,2023-01-13
...,...,...,...,...,...,...,...,...,...,...
96,QQQ,016255101,ALGN,1014868,2.433248e+08,0.163,Align Technology Inc,Common Stock,Health Care,2023-01-13
97,QQQ,98980L101,ZM,3191244,2.217595e+08,0.148,Zoom Video Communications Inc,Common Stock,Information Technology,2023-01-13
98,QQQ,98980G102,ZS,1873471,2.010234e+08,0.135,Zscaler Inc,Common Stock,Information Technology,2023-01-13
99,QQQ,76954A103,RIVN,11862081,1.951312e+08,0.131,Rivian Automotive Inc,Common Stock,Consumer Discretionary,2023-01-13


In [419]:
from etf_scraper.base import Providers
Providers.SSGA

'SSGA'

In [411]:
from importlib.resources import files


data_text = files("etf_scraper.data").joinpath("listings.csv")#.read_text()

In [412]:
pd.read_csv(data_text)

,fund_name,ticker,inception_date,benchmark,cusip,isin,exchange,item_url,fund_type,provider,product_id,asset_class,subasset_class,product_url,region,net_assets,country
0,Invesco AAA CLO Floating Rate Note ETF,ICLO,2022-12-09,NaN,46090A721,US46090A7211,CBOE,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Invesco Senior Loan ETF,BKLN,2011-03-03,SPBDLL,46138G508,US46138G5080,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Invesco Agriculture Commodity Strategy No K-1 ETF,PDBA,2022-08-24,DBLCDBAT,46090F308,US46090F3082,NASDAQ,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Invesco DB Agriculture Fund,DBA,2007-01-05,DBLCDBAT,46140H106,US46140H1068,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Invesco DB Base Metals Fund,DBB,2007-01-05,DBLCYTIM,46140H700,US46140H7008,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,iShares MSCI BIC ETF,BKF,2007-11-12,NaN,464286657,US4642866572,NaN,NaN,ETF,IShares,239614,Equity,Large/Mid Cap,https://www.ishares.com/us/products/239614/ish...,Global,8.825512e+07,Broad
1236,iShares MSCI Brazil Small-Cap ETF,EWZS,2010-09-28,NaN,464289131,US4642891315,NaN,NaN,ETF,IShares,239613,Equity,Small Cap,https://www.ishares.com/us/products/239613/ish...,Latin America,7.998925e+07,Brazil
1237,iShares ESG Advanced MSCI EAFE ETF,DMXF,2020-06-16,NaN,46436E759,US46436E7590,NaN,NaN,ETF,IShares,314362,Equity,Large Cap,https://www.ishares.com/us/products/314362/ish...,Global,4.246446e+08,Broad
1238,iShares MSCI Brazil ETF,EWZ,2000-07-10,NaN,464286400,US4642864007,NaN,NaN,ETF,IShares,239612,Equity,Large/Mid Cap,https://www.ishares.com/us/products/239612/ish...,Latin America,4.972791e+09,Brazil


In [16]:
from etf_scraper.api import query_listings, load_listings, SCRAPERS
l = query_listings()
l.to_csv('./listings.csv', index=False)

/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:113: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  resp_df_.loc[:, "net_assets"] = resp_df_["net_assets"].apply(parse_net)
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ssga_web_data_df_.loc[:, "inception_date"] = ssga_web_data_df_[
/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py:318: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the value

In [2]:
l

,fund_name,inception_date,ticker,cusip,isin,asset_class,subasset_class,country,region,product_url,product_id,net_assets,fund_type,provider,benchmark,exchange
0,iShares MSCI China ETF,2011-03-29,MCHI,46429B671,US46429B6719,Equity,Large/Mid Cap,China,Asia Pacific,https://www.ishares.com/us/products/239619/ish...,239619,9.092071e+09,ETF,Provider.IShares,NaN,NaN
1,iShares MSCI Chile ETF,2007-11-12,ECH,464286640,US4642866408,Equity,All Cap,Chile,Latin America,https://www.ishares.com/us/products/239618/ish...,239618,5.390447e+08,ETF,Provider.IShares,NaN,NaN
2,iShares MSCI EAFE ETF,2001-08-14,EFA,464287465,US4642874659,Equity,Large/Mid Cap,Broad,Global,https://www.ishares.com/us/products/239623/ish...,239623,4.885363e+10,ETF,Provider.IShares,NaN,NaN
3,iShares MSCI EAFE Growth ETF,2005-08-01,EFG,464288885,US4642888857,Equity,Large/Mid Cap,Broad,Global,https://www.ishares.com/us/products/239622/ish...,239622,1.111276e+10,ETF,Provider.IShares,NaN,NaN
4,iShares MSCI Denmark ETF,2012-01-25,EDEN,46429B523,US46429B5232,Equity,All Cap,Denmark,Europe,https://www.ishares.com/us/products/239621/ish...,239621,2.195321e+08,ETF,Provider.IShares,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,The Industrial Select Sector SPDR® Fund,1998-12-16 00:00:00,XLI,81369Y704,US81369Y7040,Equity,NaN,NaN,US,https://www.ssga.com/us/en/intermediary/etfs/f...,NaN,1.493066e+10,ETF,Provider.SSGA,Industrial Select Sector Index,NYSE ARCA EXCHANGE
206,The Materials Select Sector SPDR® Fund,1998-12-16 00:00:00,XLB,81369Y100,US81369Y1001,Equity,NaN,NaN,US,https://www.ssga.com/us/en/intermediary/etfs/f...,NaN,5.988810e+09,ETF,Provider.SSGA,Materials Select Sector Index,NYSE ARCA EXCHANGE
207,The Real Estate Select Sector SPDR® Fund,2015-10-07 00:00:00,XLRE,81369Y860,US81369Y8600,Equity,NaN,NaN,US,https://www.ssga.com/us/en/intermediary/etfs/f...,NaN,5.356750e+09,ETF,Provider.SSGA,Real Estate Select Sector Index,NYSE ARCA EXCHANGE
208,The Technology Select Sector SPDR® Fund,1998-12-16 00:00:00,XLK,81369Y803,US81369Y8030,Equity,NaN,NaN,US,https://www.ssga.com/us/en/intermediary/etfs/f...,NaN,3.930098e+10,ETF,Provider.SSGA,Technology Select Sector Index,NYSE ARCA EXCHANGE


In [14]:
a.name

'IShares'

In [3]:
ll = load_listings()
ll

,fund_name,ticker,inception_date,benchmark,cusip,isin,exchange,item_url,fund_type,provider,product_id,asset_class,subasset_class,product_url,region,net_assets,country
0,Invesco AAA CLO Floating Rate Note ETF,ICLO,2022-12-09,NaN,46090A721,US46090A7211,CBOE,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Invesco Senior Loan ETF,BKLN,2011-03-03,SPBDLL,46138G508,US46138G5080,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Invesco Agriculture Commodity Strategy No K-1 ETF,PDBA,2022-08-24,DBLCDBAT,46090F308,US46090F3082,NASDAQ,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Invesco DB Agriculture Fund,DBA,2007-01-05,DBLCDBAT,46140H106,US46140H1068,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Invesco DB Base Metals Fund,DBB,2007-01-05,DBLCYTIM,46140H700,US46140H7008,NYSE Arca,https://www.invesco.com/us/financial-products/...,ETF,Invesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,iShares MSCI BIC ETF,BKF,2007-11-12,NaN,464286657,US4642866572,NaN,NaN,ETF,IShares,239614,Equity,Large/Mid Cap,https://www.ishares.com/us/products/239614/ish...,Global,8.825512e+07,Broad
1236,iShares MSCI Brazil Small-Cap ETF,EWZS,2010-09-28,NaN,464289131,US4642891315,NaN,NaN,ETF,IShares,239613,Equity,Small Cap,https://www.ishares.com/us/products/239613/ish...,Latin America,7.998925e+07,Brazil
1237,iShares ESG Advanced MSCI EAFE ETF,DMXF,2020-06-16,NaN,46436E759,US46436E7590,NaN,NaN,ETF,IShares,314362,Equity,Large Cap,https://www.ishares.com/us/products/314362/ish...,Global,4.246446e+08,Broad
1238,iShares MSCI Brazil ETF,EWZ,2000-07-10,NaN,464286400,US4642864007,NaN,NaN,ETF,IShares,239612,Equity,Large/Mid Cap,https://www.ishares.com/us/products/239612/ish...,Latin America,4.972791e+09,Brazil


In [5]:
from etf_scraper.base import SecurityListing

a = SecurityListing(fund_name='SPY')

TypeError: SecurityListing.__new__() missing 2 required positional arguments: 'ticker' and 'provider'

In [215]:
%load_ext autoreload
%autoreload 2


from etf_scraper.api import ETFScraper

es = ETFScraper()
es.query_holdings('VOO', '2022-11-29')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValueError: No Vanguard data returned for ticker: VOO, date: 2022-11-29

In [209]:
from traceback import format_exception
from multiprocessing.pool import ThreadPool
import pandas as pd

dates = pd.bdate_range('2022-01-01', '2023-01-17')

def f(x):
    print(x)
    try:
        return es.query_holdings("IVV", x)
    except Exception as e:
        return e
    

# with ThreadPool(20) as p:
#     from_pool = p.map(f, dates)

invalid_resp = {dt: k for dt, k in zip(dates, from_pool) if not isinstance(k, pd.DataFrame)}

len(dates), len(invalid_resp)

(272, 11)

In [205]:
invalid_resp

[ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned"),
 ValueError("Found '-' as holdings date -> no data returned")]

In [210]:
invalid_resp

{Timestamp('2022-01-17 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-02-21 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-04-15 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-05-30 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-06-20 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-07-04 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-07-08 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-09-05 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-11-24 00:00:00', freq='B'): ValueError("Found '-' as holdings date -> no data returned"),
 Timestamp('2022-12-26 00:00:00', fre

In [91]:
es.listings_df[es.listings_df['ticker']=='IVV']

,fund_name,inception_date,ticker,cusip,isin,asset_class,subasset_class,country,region,product_url,product_id,net_assets,fund_type,provider,benchmark,exchange
107,iShares Core S&P 500 ETF,2000-05-15,IVV,464287200,US4642872000,Equity,Large Cap,United States,North America,https://www.ishares.com/us/products/239726/ish...,239726,3.005641e+11,ETF,IShares,NaN,NaN


In [10]:
import fsspec

fs = fsspec.filesystem('file', )

In [222]:
type(fs)

fsspec.implementations.local.LocalFileSystem

In [224]:
from fsspec.spec import AbstractFileSystem

In [49]:
%load_ext autoreload
%autoreload 2

import pandas_market_calendars as mcal

s, e = '2022-01-01', '2030-01-01'
early = mcal.get_calendar("NYSE")
date_range = mcal.date_range(early.schedule(s, e), frequency='1M')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_15484/3723384815.py:8: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  date_range = mcal.date_range(early.schedule(s, e), frequency='1M')


In [5]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [27]:
from pathlib import Path

from etf_scraper.storage import parse_holdings_filename

f = "/tmp/out/IVV_2021_09_30.csv"

Path(f).name

'IVV_2021_09_30.csv'

In [9]:
    
query_dates = get_interval_query_dates('2022-01-01', '2023-01-20', False, False)
query_tickers = ["IVV"]
    
save_func_ = partial(save_func, out_dir="/tmp/out/")

a = query_range(query_dates, query_tickers, ETFScraper(), save_func_)

INFO:etf_scraper.api:Querying for IVV holdings as of 2022-01-03 from IShares
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-01-12 from IShares
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-01-21 from IShares
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-02-01 from IShares
INFO:etf_scraper.scrapers:Querying IShares for IVV holdings as of 2022-01-03
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-02-10 from IShares
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-03-02 from IShares
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-03-11 from IShares
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-03-31 from IShares
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-03-22 from IShares
INFO:etf_scraper.api:Querying for IVV holdings as of 2022-02-21 from IShares
INFO:etf_scraper.scrapers:Querying IShares for IVV holdings as of 2022-01-12
INFO:etf_scraper.scrapers:Querying IShares for IVV holdings as of 2022-01-21

In [129]:
ETFScraper().query_holdings("IVV", '2019-04-30')

INFO:etf_scraper.api:Querying for IVV holdings as of 2019-04-30 from IShares
INFO:etf_scraper.scrapers:Querying IShares for IVV holdings as of 2019-04-30
INFO:etf_scraper.scrapers:Found reported holdings date string Apr 30, 2019
INFO:etf_scraper.scrapers:Attempting to parse holdings data


,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,MSFT,MICROSOFT CORP,Information Technology,Equity,7.279121e+09,4.07,7.279121e+09,55735996.0,130.60,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2019-04-30
1,AAPL,APPLE INC,Information Technology,Equity,6.530224e+09,3.65,6.530224e+09,32542106.0,200.67,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2019-04-30
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,5.774701e+09,3.23,5.774701e+09,2997478.0,1926.52,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2019-04-30
3,FB,FACEBOOK CLASS A INC,Communication,Equity,3.351651e+09,1.87,3.351651e+09,17330150.0,193.40,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2019-04-30
4,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Financials,Equity,3.061374e+09,1.71,3.061374e+09,14126595.0,216.71,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2019-04-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,MAC,MACERICH REIT,Real Estate,Equity,2.980255e+07,0.02,2.980255e+07,742465.0,40.14,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2019-04-30
505,UA,UNDER ARMOUR INC CLASS C,Consumer Discretionary,Equity,2.866191e+07,0.02,2.866191e+07,1383297.0,20.72,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2019-04-30
506,UBFUT,CASH COLLATERAL USD UBFUT,Cash and/or Derivatives,Cash Collateral and Margins,2.532480e+07,0.01,2.532480e+07,25324800.0,100.00,United States,NaN,USD,1.0,USD,NaN,IVV,2019-04-30
507,NWS,NEWS CORP CLASS B,Communication,Equity,1.142235e+07,0.01,1.142235e+07,914520.0,12.49,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2019-04-30


In [127]:
print(a[list(a)[0]]['error'])

Traceback (most recent call last):
  File "/tmp/ipykernel_15484/2438578564.py", line 29, in query_holdings
    save_path, n_holdings = _query_holdings(ticker, query_date)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_15484/2438578564.py", line 23, in _query_holdings
    holdings_df = etf_scraper.query_holdings(ticker, query_date)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/api.py", line 117, in query_holdings
    return scraper._retrieve_holdings(sec_listing, holdings_date)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py", line 216, in _retrieve_holdings
    return cls.retrieve_holdings(
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nikul/Documents/Programming/ETF_Scraper/src/etf_scraper/scrapers.py", line 200, in retrieve_holdings
    holdings_df, as_of_d

In [122]:
!ls -l /tmp/out/*


# !du -h /tmp/out/
# !mkdir /tmp/out/

.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2010_03_31.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2010_04_30.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2010_06_30.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2010_08_31.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2010_09_30.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2010_11_30.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2010_12_31.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2011_01_31.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2011_02_28.csv
.rw-rw-r-- 74k nikul 21 Jan 12:00 /tmp/out/IVV_2011_03_31.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2011_05_31.csv
.rw-rw-r-- 74k nikul 21 Jan 12:00 /tmp/out/IVV_2011_06_30.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2011_08_31.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2011_09_30.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2011_10_31.csv
.rw-rw-r-- 73k nikul 21 Jan 12:00 /tmp/out/IVV_2011_11_30.csv
.rw-rw-r

In [21]:
get_interval_query_dates(s, e, False, True)

array([datetime.date(2022, 1, 3), datetime.date(2022, 1, 4),
       datetime.date(2022, 1, 5), datetime.date(2022, 1, 6),
       datetime.date(2022, 1, 7), datetime.date(2022, 1, 10),
       datetime.date(2022, 1, 11), datetime.date(2022, 1, 12),
       datetime.date(2022, 1, 13), datetime.date(2022, 1, 14),
       datetime.date(2022, 1, 18), datetime.date(2022, 1, 19),
       datetime.date(2022, 1, 20), datetime.date(2022, 1, 21),
       datetime.date(2022, 1, 24), datetime.date(2022, 1, 25),
       datetime.date(2022, 1, 26), datetime.date(2022, 1, 27),
       datetime.date(2022, 1, 28), datetime.date(2022, 1, 31),
       datetime.date(2022, 2, 1), datetime.date(2022, 2, 2),
       datetime.date(2022, 2, 3), datetime.date(2022, 2, 4),
       datetime.date(2022, 2, 7), datetime.date(2022, 2, 8),
       datetime.date(2022, 2, 9), datetime.date(2022, 2, 10),
       datetime.date(2022, 2, 11), datetime.date(2022, 2, 14),
       datetime.date(2022, 2, 15), datetime.date(2022, 2, 16),
    

In [148]:
df = es.query_holdings(
    "IVV",
    "2022-10-20"
    # date(year=2022, month=10, day=30),
)
df

,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,AAPL,APPLE INC,Information Technology,Equity,1.943289e+10,7.01,1.943289e+10,135524734.0,143.39,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2022-10-20
1,MSFT,MICROSOFT CORP,Information Technology,Equity,1.580006e+10,5.70,1.580006e+10,66906870.0,236.15,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2022-10-20
2,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,9.163971e+09,3.31,9.163971e+09,79513852.0,115.25,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2022-10-20
3,GOOGL,ALPHABET INC CLASS A,Communication,Equity,5.377561e+09,1.94,5.377561e+09,53791743.0,99.97,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2022-10-20
4,TSLA,TESLA INC,Consumer Discretionary,Equity,4.952846e+09,1.79,4.952846e+09,23894470.0,207.28,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2022-10-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,UBFUT,CASH COLLATERAL USD UBFUT,Cash and/or Derivatives,Cash Collateral and Margins,3.344380e+07,0.01,3.344380e+07,33443800.0,100.00,United States,NaN,USD,1.0,USD,"Oct 01, 2015",IVV,2022-10-20
503,VNO,VORNADO REALTY TRUST REIT,Real Estate,Equity,3.234572e+07,0.01,3.234572e+07,1481031.0,21.84,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2022-10-20
504,DISH,DISH NETWORK CORP CLASS A,Communication,Equity,3.066050e+07,0.01,3.066050e+07,2201041.0,13.93,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2022-10-20
505,NWS,NEWS CORP CLASS B,Communication,Equity,1.597072e+07,0.01,1.597072e+07,944454.0,16.91,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2022-10-20


In [138]:
df

,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,GHC,GRAHAM HOLDINGS COMPANY CO CLASS B,Consumer Discretionary,Equity,4.108452e+06,0.01,NaN,12078.00,340.16,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2011-10-31
1,AZO,AUTOZONE INC,Consumer Discretionary,Equity,2.262185e+07,0.08,NaN,69909.00,323.59,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2011-10-31
2,CMG,CHIPOTLE MEXICAN GRILL INC,Consumer Discretionary,Equity,2.530715e+07,0.09,NaN,75292.00,336.12,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2011-10-31
3,SHLDQ,SEARS HOLDINGS CORP,Consumer Discretionary,Equity,7.247286e+06,0.03,NaN,92700.00,78.18,United States,Non-Nms Quotation Service (Nnqs),USD,1.0,USD,NaN,IVV,2011-10-31
4,ISRG,INTUITIVE SURGICAL INC,Health Care,Equity,4.075551e+07,0.15,NaN,93937.00,433.86,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2011-10-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,MSFT,MICROSOFT CORP,Information Technology,Equity,4.770068e+08,1.74,NaN,17912386.00,26.63,United States,NASDAQ,USD,1.0,USD,NaN,IVV,2011-10-31
497,PFE,PFIZER INC,Health Care,Equity,3.609755e+08,1.31,NaN,18742238.00,19.26,United States,New York Stock Exchange Inc.,USD,1.0,USD,NaN,IVV,2011-10-31
498,BAC,BANK OF AMERICA CORP,Financials,Equity,1.662717e+08,0.61,NaN,24344320.00,6.83,United States,SIX Swiss Exchange,USD,1.0,USD,NaN,IVV,2011-10-31
499,GEC,GENERAL ELECTRIC,Industrials,Equity,4.255053e+08,1.55,NaN,25464113.00,16.71,United States,Deutsche Boerse Xetra,USD,1.0,USD,NaN,IVV,2011-10-31


In [46]:
l[(l['provider']=='Invesco')&(l['fund_name'].str.contains('QQQ'))]

,fund_name,inception_date,ticker,cusip,isin,asset_class,subasset_class,country,region,product_url,product_id,net_assets,fund_type,provider,benchmark,exchange
138,Invesco QQQ,1999-03-10,QQQ,46090E103,US46090E1038,NaN,NaN,NaN,NaN,https://www.invesco.com/us/financial-products/...,NaN,NaN,ETF,Invesco,NDX,NASDAQ
